In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc

import numpy as np
import pandas as pd
import torch
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
from einops import rearrange
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from research.scripts.bigbigan import *

In [ ]:
dataset_path = Path('D:\\Datasets\\NSD\\')
derivatives_path = dataset_path / 'derivatives'
betas_path = dataset_path / 'nsddata_betas' / 'ppdata'
ppdata_path = dataset_path / 'nsddata' / 'ppdata'

In [ ]:
model_name = 'bigbigan-resnet50'# ResNet-50
#model_name = 'bigbigan-revnet50x4' # RevNet-50 x4
module_path = f'https://tfhub.dev/deepmind/{model_name}/1'

# module = hub.Module(module_path, trainable=True, tags={'train'})  # training
module = hub.Module(module_path)  # inference

for signature in module.get_signature_names():
    print('Signature:', signature)
    print('Inputs:', pformat(module.get_input_info_dict(signature)))
    print('Outputs:', pformat(module.get_output_info_dict(signature)))
    print()

In [ ]:
bigbigan = BigBiGAN(module)

# Make input placeholders for x (`enc_ph`) and z (`gen_ph`).
enc_ph = bigbigan.make_encoder_ph()
gen_ph = bigbigan.make_generator_ph()

# Compute samples G(z) from encoder input z (`gen_ph`).
gen_samples = bigbigan.generate(gen_ph)

# Compute reconstructions G(E(x)) of encoder input x (`enc_ph`).
recon_x = bigbigan.reconstruct_x(enc_ph, upsample=True)

# Compute encoder features used for representation learning evaluations given
# encoder input x (`enc_ph`).
enc_features = bigbigan.encode(enc_ph, return_all_features=True)

# Compute discriminator scores for encoder pairs (x, E(x)) given x (`enc_ph`)
# and generator pairs (G(z), z) given z (`gen_ph`).
disc_scores_enc = bigbigan.discriminate(*bigbigan.enc_pairs_for_disc(enc_ph))
disc_scores_gen = bigbigan.discriminate(*bigbigan.gen_pairs_for_disc(gen_ph))

# Compute losses.
losses = bigbigan.losses(enc_ph, gen_ph)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
feed_dict = {gen_ph: np.random.randn(12, 120)}
_out_samples = sess.run(gen_samples, feed_dict=feed_dict)
print('samples shape:', _out_samples.shape)
imshow(imgrid(image_to_uint8(_out_samples), cols=4))

In [ ]:
subjects = {f'subj0{i}': {} for i in range(1, 9)}

for subject_name, subject_data in subjects.items():
    responses_file_path = ppdata_path / subject_name / 'behav' / 'responses.tsv'
    subject_data['responses'] = pd.read_csv(responses_file_path, sep='\t',)
    
    # The last 3 sessions are currently held-out for the algonauts challenge
    # remove them for now.
    session_ids = subject_data['responses']['SESSION']
    held_out_mask = session_ids > (np.max(session_ids) - 3)
    subject_data['responses'] = subject_data['responses'][~held_out_mask]

In [ ]:
stimulu_path = dataset_path / 'nsddata_stimuli' / 'stimuli' / 'nsd' / 'nsd_stimuli.hdf5'
stimulus_images = h5py.File(stimulu_path, 'r')['imgBrick']

subject_name = 'subj01'

split_name = 'split-01'
split = h5py.File(derivatives_path / 'data_splits' / f'{split_name}.hdf5')
subject_split = split[subject_name]

test_response_mask = subject_split['test_response_mask'][:].astype(bool)
validation_response_mask = subject_split['validation_response_mask'][:].astype(bool)
training_response_mask = ~(test_response_mask | validation_response_mask)

responses = subjects[subject_name]['responses']
response_stimulus_ids = responses['73KID'].to_numpy()


def select_fold_images(stimulus_images, response_mask, response_stimulus_ids):
    response_ids = np.where(response_mask)[0]
    stimulus_ids = response_stimulus_ids[response_ids] - 1
    
    argsort_ids = np.argsort(stimulus_ids)
    unique_results = np.unique(stimulus_ids, return_counts=True)
    unique_stimulus_ids, unique_stimulus_counts = unique_results
    
    X_averaged = stimulus_images[unique_stimulus_ids]
    X = np.repeat(X_averaged, unique_stimulus_counts, axis=0)
    
    return X, X_averaged

X_val, X_val_avg = select_fold_images(stimulus_images, validation_response_mask, response_stimulus_ids)

In [ ]:
X_val.shape, X_val_avg.shape

In [ ]:
embedding_model = 'bigbigan-resnet50'
embedding_key = 'z_mean'
code_name = 'averaging'
version = '1-0'

in_path = derivatives_path / 'decoded_features' / embedding_model / embedding_key / subject_name
Z_val_avg_pred = np.load(in_path / f'Y_val_avg_pred__{code_name}__v{version}.npy')
Z_val_avg = np.load(in_path / f'Y_val_avg__{code_name}__v{version}.npy')
Z_val_pred = np.load(in_path / f'Y_val_pred__{code_name}__v{version}.npy')
Z_val = np.load(in_path / f'Y_val__{code_name}__v{version}.npy')


In [ ]:
Z_val_pred[:, 0].std()

In [ ]:
from scipy import ndimage, misc
from PIL import Image

out_path = derivatives_path / 'results' / embedding_model / embedding_key / code_name / version / subject_name
out_path.mkdir(exist_ok=True, parents=True)

def make_images(x, z, z_pred):
    num_samples = 13
    z_pred_mean = z_pred[0]
    z_pred_std = z_pred[1]
    sample = np.random.randn(num_samples, z_pred_mean.shape[0])
    z_pred = z_pred_mean[None, :] + z_pred_std[None, :] * sample

    x_pred = sess.run(gen_samples, feed_dict={gen_ph: z_pred})
    x_recon = sess.run(gen_samples, feed_dict={gen_ph: z[None]})
    x = ndimage.zoom(x, (128 / x.shape[0], 128 / x.shape[1], 1))
    x = x / 256 * 2 - 1

    out = np.concatenate([x[None], x_recon, x_pred])

    img = imgrid(image_to_uint8(out), cols=5)
    return img

def save_images(X, Z, Z_pred):
    for i in tqdm(range(X.shape[0])):
        img = make_images(X[i], Z[i], Z_pred[i])
        Image.fromarray(img).save(out_path / f'img{i:04}.png')
        
def view_images(X, Z, Z_pred):
    @interact(i=(0, X.shape[0]-1), scale=(0., 2.))
    def show(i, scale):
        z = Z_pred[i] * scale
        print(z.std())
        img = make_images(X[i], Z[i], z)
        
        plt.figure(figsize=(12, 12))
        plt.imshow(img)
    
#view_images(X_val_avg, Z_val_avg, Z_val_avg_pred)
#view_images(X_val, Z_val, Z_val_pred)
save_images(X_val, Z_val, Z_val_pred)

In [ ]:
feed_dict = {gen_ph: np.random.randn(12, 120)}
_out_samples = sess.run(gen_samples, feed_dict=feed_dict)
print('samples shape:', _out_samples.shape)
imshow(imgrid(image_to_uint8(_out_samples), cols=4))

In [ ]:
_out_samples.shape